In [ ]:
# Importamos las librerías

%matplotlib inline

from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn.metrics import r2_score

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm

In [ ]:
# Importamos el dataset

df = pd.read_csv('./data/data_limpio_gdf_caba.csv')

print(df.shape)
df.head()

In [ ]:
# Seleccionamos las columnas que vamos a utilizar y sólo las propiedades de tipo apartamento

df = df[df['tipo'] == 'apartment']
df = df[['municipio', 'sup_m2_total', 'ambientes_cat', 'lat', 'lon', 'precio_usd' ]]
df.head()

In [ ]:
# Extraemos un dataframe por cada columna

df_sup_m2_total = df[['sup_m2_total', 'precio_usd', 'municipio']]
df_ambientes_cat = df[['ambientes_cat', 'precio_usd', 'municipio']]
df_ubica = df[['lat', 'lon', 'precio_usd', 'municipio']]

In [ ]:
# Creamos las variables dummies para la variable categórica de ambientes

df_ambientes_cat = pd.get_dummies(df_ambientes_cat, columns=['ambientes_cat'], drop_first=True)
print(df_ambientes_cat.shape)
df_ambientes_cat.head()

In [ ]:
# Armamos la función para las regresiones lineales

def regresion_lineal(df_lr, target, test_size=0.2, random_state=42):
    # Separamos la variable objetivo del resto del dataset
    X = df_lr.drop(target, axis=1)
    y = df_lr[target]
    
    # Dividimos el dataset en train y test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # Normalizamos los datos
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Instanciamos el modelo
    model = linear_model.LinearRegression()
    
    # Entrenamos el modelo
    model.fit(X_train, y_train)
    
    # Predecimos con el modelo
    y_pred = model.predict(X_test)
    
    # Calculamos el R2
    r2 = r2_score(y_test, y_pred)
    
    # Imprimimos los resultados
    print('R2: ', r2)
    print('Intercept: ', model.intercept_)
    print('Coeficiente: ', model.coef_[0])
    
    # Graficamos los resultados
    plt.figure(figsize=(4, 3))
    plt.scatter(y_test, y_pred)
    plt.axline(xy1=(0, 0), slope=1, color='red')
    plt.xlabel('Valores reales')
    plt.ylabel('Valores predichos')
    plt.xlim(0, 1000000)
    plt.ylim(0, 1000000)
    plt.show()


In [ ]:
# Creo un array con los municipios

municipios = df['municipio'].unique()

In [ ]:
# Regresión lineal para el precio de las propiedades en función del municipio y la superficie total y guardamos el modelo en un mapa

for municipio in municipios:
    df_municipio = df_sup_m2_total[df_sup_m2_total['municipio'] == municipio]
    if(df_municipio.shape[0] > 100):
        df_municipio = df_municipio.drop('municipio', axis=1)
        print('Municipio: ', municipio)
        regresion_lineal(df_municipio, 'precio_usd')
        print('----------------------------------------')
